In [48]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np

In [ ]:
##Load the trained model,scaler pickle, onehot
model = load_model('model.h5')

##Load the encoder and scaler
with open('label_encode_gender.pkl', 'rb') as file:
    label_encode_gender = pickle.load(file)

with open('ohe_geo.pkl', 'rb') as file:
    label_encode_geo = pickle.load(file)

with open('scaler.pkl', 'rb') as file:
    scaler = pickle.load(file)

In [79]:
label_encode_geo.categories_[0]

array(['France', 'Germany', 'Spain'], dtype=object)

In [ ]:
label_encode_gender.classes_

array(['Male'], dtype=object)

In [64]:
##INPUT DATA 
input_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumofProduct': 2,
    'HasCrCredit': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}

In [60]:
##One-hot encode 'Geography'
geo_encoded = label_encode_geo.transform([[input_data['Geography']]]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded,columns=label_encode_geo.get_feature_names_out(['Geography']))
geo_encoded_df

c:\Users\Neosoft\Documents\Projects\Gen AI ANN Project\venv\Lib\site-packages\sklearn\utils\validation.py:2691: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [ ]:
input_df = pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumofProduct,HasCrCredit,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [66]:
input_df['Gender'] = label_encode_gender.fit_transform(input_df['Gender'])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumofProduct,HasCrCredit,IsActiveMember,EstimatedSalary
0,600,France,0,40,3,60000,2,1,1,50000


In [67]:
#Concat one hot encode data with original data
input_df = pd.concat([input_df.drop(['Geography'], axis=1),geo_encoded_df],axis=1)
input_df.head()

,CreditScore,Gender,Age,Tenure,Balance,NumofProduct,HasCrCredit,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,0,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [68]:
##Scaling the input data
input_scaled = scaler.transform(input_df)
input_scaled

array([[-0.32443942, -1.        , -0.04878855, -0.80003628, -0.3560182 ,
         0.26726124,  0.57735027,  1.        , -1.43376327,  1.1055416 ,
        -0.65465367, -0.57735027]])

In [ ]:
prediction = model.predict(input_scaled)
prediction

1/1 [==============================] - 1s 577ms/step


array([[0.2086626]], dtype=float32)

In [70]:
prediction_prob = prediction[0][0]

In [71]:
prediction_prob

0.2086626

In [73]:
if prediction_prob > 0.5:
    print("Customer will surely churn")
else: 
    print("Customer will not churn")

Customer will not churn
